In [5]:
!pip install openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.5/473.5 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.3/310.3 kB 10.5 MB/s eta 0:00:00


In [2]:
import os
import re
import time
import numpy as np
import pandas as pd
from openai import OpenAI

In [4]:
products_df = pd.read_csv("data/labeled_appliances.csv")


In [6]:
len(products_df)

7445

In [8]:
products_df.head()

,id,title,description,label
0,B07W42P978,WD12X10327 Rack Roller and stud assembly Kit (...,PARTS NUMBERThe WD12X10327 dishwasher top rack...,29
1,B083Q6Y54F,G.a HOMEFAVOR Cold Brew Coffee Infuser 64oz (2...,COMPLETE COLD BREW SYSTEM The Cold Brew Kit is...,19
2,B00KAS9ZMG,3-pack OnePurify Water Filter Replacement Cart...,This OnePurify refrigerator water filter conta...,0
3,B01AQHSNSM,SAMSUNG DA97-11092B Genuine OEM Ice Maker Asse...,This high quality Genuine OEM Samsung Ice Make...,32
4,B0BWCL9WCH,PARTY BARGAINS 600 Paper Coffee Filters - Whit...,PARTY BARGAINS Disposable Coffee Filters Sing...,7


In [10]:
products_df.iloc[4333]

id                                                    B085YBLD9V
title          (New) 316048413 316048414 Genuine OEM compatib...
description    316048413 AP2124664 08002898 08004682 08005370...
label                                                         30
Name: 4333, dtype: object

In [12]:
products_df = products_df.drop(index=4333).reset_index(drop=True)


In [14]:
len(products_df)

7444

In [16]:
# sampling 5 products
product_samples = products_df.groupby('label').sample(n=5, random_state=234).reset_index(drop=True)
product_samples.head(n=12)

,id,title,description,label
0,B07QK6L51Z,Pack of 24 Replacement Charcoal Water Filters ...,24PACK OF PREMIUM QUALITY REPLACEMENT CHARCOAL...,-1
1,B00W4YHLU2,Supco SM279827 Dryer Drive Motor Replaces Whir...,Supco SM279827 Dryer Drive Motor This highqual...,-1
2,B000XXWOGW,"Supco ES671 Washer Lid Switch, Replaces Whirlp...",Replacement Washer Lid SwitchThis washer lid s...,-1
3,B08DTLLFG6,AQUALINK DA29-00020B Replacement for Samsung H...,NSFANSI 42 Standard tested verified by indepe...,-1
4,B0BWYG6YVZ,Ultra Durable 74004053 Range Top Burner Ignite...,Customer satisfaction is our priority If for a...,-1
5,B07176DGS9,"Waterdrop Refrigerator Water Filter, Replaceme...",Note WF2CB Double check the exact model number...,0
6,B078VZYJM5,Filterlogic DA29-00003G Refrigerator Water Fil...,Certified to NSF 42to significantly improve wa...,0
7,B00VVMKIWQ,GE MWF Refrigerator Water Filter Compatible Re...,AQUA BLUE GE MWF MWFA refrigerator water filte...,0
8,B01LZR75SY,K&J Refrigerator Water Filter Replacement Mayt...,Highest Quality Filtration The activated carb...,0
9,B0793S6X7C,"ICEPURE 10""X2.5"" 5 Microns Whole House Sedimen...",NSF 42 CERTIFIEDTested and certified by WQA ag...,0


In [18]:
product_samples = product_samples.sample(frac=1).reset_index(drop=True)
product_samples.head()

,id,title,description,label
0,B08NSJZSNJ,"W11165546, 33090105, W10599423 Washer Water In...",Package Includes 1 W11165546 Washer Water Inl...,42
1,B0BWYG6YVZ,Ultra Durable 74004053 Range Top Burner Ignite...,Customer satisfaction is our priority If for a...,-1
2,B07T41M9SN,KSTK1 27-inch Laundry Stacking Kit Compatible ...,PARTS NUMBERThe kstk1 lg stacking kit comprise...,3
3,B07XP2J7MG,4681EA2001T Washer Drain Pump Motor by AMI PAR...,PART NUMBER 4681EA2001T LG 4681EA2001T Drain P...,23
4,B07RBSYMM7,AMI PARTS WE1M654 Timer Knob with Metal Ring f...,Parts Number Timer Control Knob Dryer Knob WE1...,39


In [20]:
openai_api_key = os.environ['OPENAI_API_KEY']

In [22]:
class QueryGenerator:
    def __init__(self, api_key):
        """Initialize the generator with API key and standard prompts"""
        self.client = OpenAI(api_key=api_key)
        
        # Store the system and example prompts
        self.system_prompt = """You are an expert in e-commerce search optimization. Given a product description, generate a list of 5 realistic search queries a customer might use to find the product, even if the customer does not know the exact name of the product they are looking for.

        Guidelines:
        - Keep queries short (2-10 words)
        - Avoid exact repetition of product specs; focus on what a customer would type
        - Include a mix of general, descriptive, and feature-focused queries
        - Consider different use cases for each product (e.g. baking soda can be used for cooking, cleaning, and removing bad odors).
        - Use casual, natural language
        - Return ONLY the list of 5 queries enclosed in [] and separated by ',' with no additional text"""

        self.example_prompt = """Here are some examples:

        Product Description: "Simplify your plumbing projects with the Eastman ArmorFlex dryer vent These replacement dryer vents are constructed of semirigid aluminum for thermal resistance and hold their shape so that you can easily install it without kinking or crushing This wall dryer vent allows for 90 degree bends in tight installations and can withstand a temperature range of 22 degrees F to 482 degrees F Rated as Fire Class0 ArmorFlex is fire resistant and will not spread flames or smoke making the ArmorFlex dryer vent hose safe to use in your home DRYER VENT TUBING Dryer vent tube is designed to provide a reliable connection between your dryer and vent FLAME RESISTANT The inside dryer vent hose can withstand a temperature range of 22 degrees F to 482 degrees F Rated as Fire Class0 ArmorFlex is fire resistant and will not spread flames or smoke making the ArmorFlex dryer vent hose safe to use in your home EASY INSTALLATION Dryer vent line features a durable and extremely flexible ArmorFlex construction allowing for 90 degree bends in tight installations includes 2 stainless steel hose clamps for a steady connection DURABLE Washer and dryer hoses are constructed of semirigid aluminum for thermal resistance recommended over traditional foil ducts by dryer manufacturersArmorFlex holds its shape allowing for an easier installation while positioning your dryer into place without kinking or crushing SPECS Indoor dryer vent hose measures 8 feet long fully extended x 4 inches diameter"
        
        Queries:
        [easy install dryer vent, replacement tube for dryer air venting system, best dryer vent hose for tight spaces, replacement for crushed dryer vent tube, home dryer vent plumbing broke fix]
        
        Product Description: "Aerolattes number 2size Filter Cone is designed to brew 2 to 6cups of coffee using the manual pourover coffee method Making freshbrewed coffee in smaller amounts allows for a more enjoyable aromatic and flavorful brew with less waste No more stale or burnt coffee made hours ahead The pourover coffee method is a timetested Japanese technique that offers complete control over the quality and flavor of brewed coffee Pouring a thin slow steady stream of water over the ground coffee allows for a distinct flavorful result every time Aerolattes Filter Cone is the coffee accessory for making pourover coffee Simply rinse the Coffee Filter Cone with hot water to warm it thoroughly Place it over a mug or carafe and insert a paper or reusable coffee filter Measure in ground coffee wet them slightly and allow them to stand for 15 to 30 seconds The grounds will expand slightly and release more flavor Pour the remaining hot water slowly and evenly Allow the coffee dripper to finish dripping Remove the coffee filter cone from the mug and discard the wet grounds Drink the freshbrewed coffee immediately Aerolattes Filter Cone makes a perfect coffee accessories gift for athome baristas and coffee enthusiasts Made from finequality ceramic with a scratchfree glaze for durability Reusable coffee filter cone is dishwasher safe for easy cleanup Aerolattes Number 2Size Coffee Filter Cone is designed to brew 2 to 6cups of coffee using the manual pourover coffee method Made from finequality ceramic with a scratchfree glaze for durability Brew fresh single cup coffee with each brew no more hourold coffee thats turned stale or burnt brew only whats needed for less waste Reusable coffee filter cone perfect for the coffee enthusiast and athome baristas Easy to use and clean dishwasher safe"
        
        Queries:
        [best ceramic pour-over coffee dripper, gift for friend who loves coffee, brew coffee device manually good flavor, alternative to coffee machine, make coffee no machine pour water]
        """

    def generate_single(self, product_description):
        """Generate queries for a single product description"""
        try:
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": self.example_prompt},
                    {"role": "user", "content": f"Now generate queries for this product description: {product_description}"}
                ],
                temperature=0.7,
                max_tokens=50
            )
            
            res = response.choices[0].message.content
            return res.replace('[', '').replace(']', '').split(', ')
            
        except Exception as e:
            print(f"Error generating queries: {e}")
            return []

    def generate_batch(self, product_descriptions, batch_size: int = 5):
        """
        Generate queries for multiple products efficiently.
        Uses batching and includes rate limiting.
        """
        batch_results = []
        
        # Process in batches
        for i in range(0, len(product_descriptions), batch_size):
            batch = product_descriptions[i:i + batch_size]
            
            # Create a single prompt for the batch
            batch_prompt = "Generate queries for each of these products. For each product, start with 'Product X:' where X is the product number:\n\n"
            for idx, desc in enumerate(batch, 1):
                batch_prompt += f"Product {idx}: {desc}\n\n"
            
            try:
                response = self.client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=[
                        {"role": "system", "content": self.system_prompt},
                        {"role": "user", "content": self.example_prompt},
                        {"role": "user", "content": batch_prompt}
                    ],
                    temperature=0.7,
                    max_tokens=50 * len(batch)  # Adjust tokens based on batch size
                )
                
                # Parse the batch response
                response_text = response.choices[0].message.content
                response_list = re.sub(r"Product \d: ", "", response_text).replace('[', '').replace(']', '').split('\n')
                batch_results.extend([text.split(', ') for text in response_list])
                                
                # Rate limiting - wait 1 second between batches
                if i + batch_size < len(product_descriptions):
                    time.sleep(5)
                    
            except Exception as e:
                print(f"Error processing batch starting at index {i}: {e}")
                continue
        
        return batch_results

In [24]:
generator = QueryGenerator(openai_api_key)

In [26]:
product_samples['description'].shape

(250,)

In [28]:
results = generator.generate_batch(product_samples['description'])
print(f"Check: {results[:5]}")

Check: [['Whirlpool washing machine water inlet valve replacement', 'fix washing machine water temperature error', 'how to install Maytag washer water valve', 'Kenmore washing machine water supply issue', 'Whirlpool error code F8 E1 LF LO'], ['High compatibility oven igniter replacement part', 'fit most Whirlpool range models', 'premium quality stove igniter replacement', 'fix gas range igniter not lighting', 'how to replace oven igniter'], ['LG stacking kit for 27-inch washer and dryer', 'save space with laundry stacking kit', 'how to stack Samsung washer and dryer', 'durable washer dryer stacking support', 'LG stacking kit installation'], ['LG washing machine drain pump replacement', 'fix washer not draining issue', 'compatible with LG washer models', 'premium quality washer pump replacement', 'easy DIY install washer pump'], ['GE dryer timer control knob replacement', 'white dryer knob set', 'fix dryer timer setting issue', 'how to replace dryer control knob', 'durable timer knob fo

In [30]:
len(results)

353

In [32]:
[result for result in results if len(result[0]) <= 0]

[[''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 ['']]

In [34]:
cleaned_results = [result for result in results if len(result[0]) > 0]
len(cleaned_results)

279

In [36]:
np.unique([len(result) for result in cleaned_results])

array([1, 2, 4, 5, 6, 7])

In [38]:
inspect_index = [i for i in range(len(cleaned_results)) if len(cleaned_results[i]) != 5]
inspect_index

[19,
 46,
 48,
 50,
 52,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 218,
 226,
 228,
 230,
 232,
 235,
 277]

In [40]:
[cleaned_results[i] for i in inspect_index]

[['pour-over coffee filter for Chemex',
  'Hario V60',
  'and more',
  'beautifully designed coffee dripper',
  'mesh filter for optimal coffee extraction',
  'sealed tip coffee filter',
  'easy to clean cone filter'],
 ['        '],
 ['        '],
 ['        '],
 ['        '],
 ['Samsung Laundry Pedestal',
  '27 inch washer dryer lift',
  'Samsung laundry room storage drawer',
  'compatible with Samsung Front Load',
  'buy Samsung laundry pedestal',
  'elevate washer and dryer'],
 ['        '],
 ['Egg storage box with lid',
  'stackable refrigerator egg tray',
  '60 egg capacity container',
  'egg groove design protection',
  'store eggs in pantry',
  'portable egg holder for camping'],
 ['        '],
 ['Stainless steel frothing pitcher',
  '33 oz milk steaming pitcher',
  'durable frother pitcher',
  'froth milk for cappuccinos',
  'latte art pitcher',
  'dishwasher safe milk pitcher'],
 ['        '],
 ['Reusable Vertuoline coffee capsule',
  'stainless steel coffee seal',
  'compati

In [42]:
cleaned_results[46]

['        ']

In [44]:
del cleaned_results[46]

In [328]:
inspect_index = [i for i in range(len(cleaned_results)) if len(cleaned_results[i]) != 5]
inspect_index

[]

In [326]:
del cleaned_results[153]

In [330]:
[cleaned_results[i] for i in inspect_index]

[]

In [222]:
cleaned_results[204]

['Whirlpool dryer heating element 279838',
 'dryer parts kit for Kenmore',
 'Maytag',
 'Amana',
 'thermal fuse and thermostat kit',
 'dryer heating element replacement']

In [224]:
cleaned_results[204] = ['Whirlpool dryer heating element',
 'dryer parts kit for Kenmore Maytag Amana',
 'thermal fuse and thermostat kit',
 'dryer heating element replacement', 'stuff to fix dryer heating']

In [226]:
cleaned_results[204]

['Whirlpool dryer heating element',
 'dryer parts kit for Kenmore Maytag Amana',
 'thermal fuse and thermostat kit',
 'dryer heating element replacement',
 'stuff to fix dryer heating']

In [152]:
cleaned_results[73] = cleaned_results[73][:5]

In [154]:
cleaned_results[73]

['Reusable Vertuoline coffee capsule',
 'stainless steel coffee seal',
 'compatible with Nespresso machines',
 'make strong coffee crema',
 'reusable coffee pod filter']

In [200]:
del cleaned_results[151]

In [332]:
print(f"Number of results: {len(cleaned_results)}")
print(f"Number of queries per result: {np.unique([len(result) for result in cleaned_results])}")

Number of results: 248
Number of queries per result: [5]


In [338]:
product_samples = product_samples.iloc[:len(cleaned_results)]  # Trim to match cleaned_results
product_samples.loc[:, ['query_1', 'query_2', 'query_3', 'query_4', 'query_5']] = cleaned_results



In [340]:
product_samples.loc[:, ['query_1', 'query_2', 'query_3', 'query_4', 'query_5']] = cleaned_results

In [346]:
product_samples.sample(5)

,id,title,description,label,query_1,query_2,query_3,query_4,query_5
48,B0CFLQ4FM6,"Nugget Ice Maker Countertop, Crushed Chewable ...",FAST EFFICIENT ICE MAKER With a water tank ca...,2,chewable nugget ice maker,fast ice maker 34 pounds/day,quiet countertop ice maker,self-cleaning portable ice maker,chewable nugget ice for drinks
10,B07Z4999N2,Jacent Fill N Brew Disposable #2 Coffee Filter...,A good coffee filter is important for a great ...,34,cone shaped coffee filters,disposable coffee filters for electric coffee ...,compostable paper coffee filters,coffee maker filter replacement,coffee filters for 1 cup coffee maker
50,B07XXTW9P6,AMI PARTS WB44T10010 Range Oven Stove Bake Hea...,PART SPECIFICATIONS AMI PARTS WB44T10010 Range...,13,GE oven heating element replacement,stove bake heating element 240V,oven stove heating element 2585W,GE WB44T10010 heating element,durable high quality stove element
91,B078JB36V5,OCTOPUS 131970 W10447783 & W10435302 Bearing I...,OCTOPUS PARTS APPLIANCES PART 131970 W104477...,16,metal construction replacement tool kit,find and replace appliance parts easily,full metal bearing driver tool set,compatible with various appliance models,durable metal construction appliance parts
195,B08P6WZ791,WP8562043 8531233 Dishwasher Silverware Basket...,Highquality and durable ABS material DISHWASHE...,33,upgraded black stainless stove knob replacement,durable oven knob for Samsung range,replacement knob for NX58M6850SG,reinforced D ring stove knob,high-quality stove knob replacement


In [352]:
product_samples.to_csv('data/appliances_with_queries.csv', index=False)